In [1]:
import pandas as pd
import numpy as np

In [8]:
victims_full_info = pd.read_csv('../data/CSV_georefer/victimas_full_info.csv')

georeference_info = pd.read_csv('../data/CSV_georefer/siniestros_georef.csv',usecols=['CODIGO_ACCIDENTE','LATITUD','LONGITUD'])

manually_georeferenced_accidents = pd.read_csv('../data/CSVs/total_encontrados.csv',usecols=['CODIGO_ACCIDENTE','DIR_ALTERN','geometry','centroids'])

In [9]:
complete_victims_dataset = pd.merge(victims_full_info,georeference_info,on='CODIGO_ACCIDENTE',how='left')

In [11]:
no_georeference = complete_victims_dataset.loc[complete_victims_dataset['LONGITUD'].isnull()]

In [12]:
complete_victims_dataset = complete_victims_dataset[~complete_victims_dataset['CODIGO_ACCIDENTE'].isin(no_georeference['CODIGO_ACCIDENTE'])]

In [14]:
import geopandas as gpd

In [15]:
new_georeferenced_accidents = pd.merge(no_georeference,manually_georeferenced_accidents,on='CODIGO_ACCIDENTE',how='inner')

new_georeferenced_accidents = gpd.GeoDataFrame(new_georeferenced_accidents)

new_georeferenced_accidents = new_georeferenced_accidents.drop('geometry',axis=1)

new_georeferenced_accidents['centroids'] = gpd.GeoSeries.from_wkt(new_georeferenced_accidents['centroids'])

In [16]:
new_georeferenced_accidents['LATITUD'] = new_georeferenced_accidents['centroids'].y

new_georeferenced_accidents['LONGITUD'] = new_georeferenced_accidents['centroids'].x

In [17]:
new_georeferenced_accidents = pd.DataFrame(new_georeferenced_accidents).drop('centroids',axis=1)

In [18]:
new_georeferenced_accidents = pd.DataFrame(new_georeferenced_accidents).drop('DIR_ALTERN',axis=1)

In [19]:
complete_victims_dataset = pd.concat([complete_victims_dataset,new_georeferenced_accidents])

In [23]:
complete_victims_dataset

,CODIGO_ACCIDENTE,VEHICULO,AFECTADO,CLASE_x,COMBINACION,LOCALIDAD,CODIGO_LOCALIDAD,FECHA_HORA,LATITUD,LONGITUD
1,4401421,0,2,Peaton,Automovil,KENNEDY,KENNEDY,2015-02-01 11:15:00,4.646739,-74.149576
2,4401421,4401421-1,0,Automovil,Peaton,KENNEDY,KENNEDY,2015-02-01 11:15:00,4.646739,-74.149576
3,4401422,4401422-1,0,Automovil,Bicicleta,ENGATIVA,ENGATIVA,2015-02-01 21:30:00,4.715298,-74.117039
4,4401422,4401422-2,1,Bicicleta,Automovil,ENGATIVA,ENGATIVA,2015-02-01 21:30:00,4.715298,-74.117039
5,4401423,0,1,Peaton,Automovil,ENGATIVA,ENGATIVA,2015-01-01 09:45:00,4.684126,-74.103029
...,...,...,...,...,...,...,...,...,...,...
4508,10528273,10528273-2,0,Automovil,Moto,NaN,TEUSAQUILLO,2020-02-12 10:15:00,4.652994,-74.103193
4509,10529992,0,1,Peaton,Automovil,NaN,TEUSAQUILLO,2020-12-25 16:01:00,4.624082,-74.079099
4510,10529992,10529992-1,1,Automovil,Peaton,NaN,TEUSAQUILLO,2020-12-25 16:01:00,4.624082,-74.079099
4511,10530410,0,1,Peaton,Automovil,NaN,LOS MARTIRES,2020-12-30 12:30:00,4.616439,-74.085814


In [21]:
complete_victims_dataset[(complete_victims_dataset['CLASE_x'] == 'Peaton') & (complete_victims_dataset['COMBINACION'] == 'Mismo')]

,CODIGO_ACCIDENTE,VEHICULO,AFECTADO,CLASE_x,COMBINACION,LOCALIDAD,CODIGO_LOCALIDAD,FECHA_HORA,LATITUD,LONGITUD
71,4401487,0,1,Peaton,Mismo,SUBA,SUBA,2015-04-01 22:00:00,4.699367,-74.055835
375,4401926,0,1,Peaton,Mismo,USAQUEN,USAQUEN,2015-12-01 21:40:00,4.801728,-74.037704
389,4401984,0,1,Peaton,Mismo,SAN CRISTOBAL,SAN CRISTOBAL,2015-09-01 15:20:00,4.566500,-74.097200
466,4402080,0,1,Peaton,Mismo,TEUSAQUILLO,TEUSAQUILLO,2015-01-14 19:00:00,4.625247,-74.083669
502,4402117,0,1,Peaton,Mismo,CIUDAD BOLIVAR,CIUDAD BOLIVAR,2015-01-14 23:40:00,4.546600,-74.151200
...,...,...,...,...,...,...,...,...,...,...
4088,10481856,0,1,Peaton,Mismo,NaN,USAQUEN,2019-12-04 16:30:00,4.713347,-74.052996
4101,10482153,0,1,Peaton,Mismo,NaN,TUNJUELITO,2019-04-17 12:30:00,4.646639,-74.130233
4132,10483219,0,1,Peaton,Mismo,NaN,TUNJUELITO,2019-02-05 00:30:00,4.575837,-74.120350
4359,10491147,0,1,Peaton,Mismo,NaN,LA CANDELARIA,2019-07-20 00:05:00,4.602364,-74.076684


In [17]:
complete_victims_dataset.to_csv('../data/CSV_georefer/complete_victims_dataset.csv', encoding='utf-8', index=False)